# Convolution and Polling
полезные ссылки:
- https://youtu.be/vVaRhZXovbw
- https://androidkt.com/explain-pooling-layers-max-pooling-average-pooling-global-average-pooling-and-global-max-pooling/
---
---

# Convolution
----

## 1. Conv1D vs Conv2D vs Conv3D
---

#### Conv1D (фильтр/ядро скользит в одном измерении)
- Conv1D фиксирует только одну из двух корреляций (вертикальную или горизонтальную), что дает гораздо более ограниченную информацию.
![-](data/conv1d_il.jpg)
----
#### Conv2D (фильтр/ядро скользит в двух измерениях)
- Conv2D может быть "разложен" на два блока Conv1D, объединение вертикального Conv1D и горизонтального Conv1D фиксирует пространственную корреляцию по обеим осям. Это приемлемый подход к классификации изображений в качестве альтернативы Conv2D.
![-](data/conv2d_il.gif)
----
#### Conv3D (фильтр/ядро скользит в трех измерениях)
- Conv3D в основном используется с данными трехмерного изображения. Такие какМагнитно-резонансная томография(МРТ) данные. Данные МРТ широко используются для исследования головного мозга, спинного мозга, внутренних органов и многих других.Компьютерная томография(КТ) Сканирование также является примером трехмерных данных, которые создаются путем объединения серии рентгеновских изображений, полученных под разными углами вокруг тела. Мы можем использовать Conv3D для классификации этих медицинских данных или извлечения из них функций.
![-](data/conv3d_il.png)
----
### Вывод:
Одной из задач свертки является показать зависимость соседних клеток, которые и формируют картинку. 1D Convolution не подойдет, потому что такой вариант свертки покажет зависимость соседних пикселей только в одной оси. 3D Convolution не уместен для данного датасета, потому что у нас изображения размером (32, 32, 3). 2D Convolution лучше всего подходит для классификации rgb изображений
### `В проекте используем только 2D Convolution`
----
----

## 2. Convolution types
----

### Simple Convolution
[tf.keras.layers.Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

![](data/conv2d1.gif)
![](data/conv2d2.gif)

Много рассказывать про данный вариант свертки не буду, мы использовали его в семинарах

Можно сказать, что такой вариант свертки показал себя хорошо на датасете mnist, который очень отдальенно напоминает svhn.

#### `Такой вариант свертки мы будем использовать`

----

### Transpose Convolution
[tf.keras.layers.Conv2DTranspose](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose)

![-](data/convtranspose.jpg)

Данный вариант свертки увеличивает входные данные с помощью фильтра, размер которого больше входного слоя.

В интернете не нашел ни один вариант использования данной свертки без других типов convolution. Как я понял, такая свертка является доплнительным инстурментом.

Примером является U-net.

### `Использовать будем, но при дальнейшем изучении skip-connection и U-net подобных сетей`
----

### Convolution LSTM
[tf.keras.layers.ConvLSTM2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ConvLSTM2D)

#### Сетевая структура, способная улавливать пространственно-временные корреляции, а именно ConvLSTM. В Keras это отражено в классе ConvLSTM2D, который вычисляет сверточные операции как во входных данных, так и в рекуррентных преобразованиях.
Это больше используется для видео (временная корреляция изображений)
#### `Мы это использовать не будем`
----

### Depthwise Convolution
[tf.keras.layers.DepthwiseConv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/DepthwiseConv2D)

![-](data/depthwise_conv.gif)

Для svhn данный вариант свертки будет работать так:
- Каждый входящий слой (глубина каждого цвета) будет делиться на три отдельных слоя
- Для каждого слоя будет создан фильтр размера (n, n, 1)
- Каждый входной слой будет "прогоняться" через свой фильтр

Плюсы:
- Уменьшаем кол-во вычеслений (сильно)
- Хорошо показываем зависимость соседних пикселей

Минусы:
- Мы не учитываем зависимость между слоями

#### `Проверим на практике работоспособность такого варианта свертки`
----

### Separable Convolution
[tf.keras.layers.SeparableConv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/SeparableConv2D)

![-](data/separable_conv.gif)

Для svhn данный вариант свертки будет работать так:
- Каждый входящий слой (глубина каждого цвета) будет делиться на три отдельных слоя
- Для каждого слоя будет создан фильтр размера (n, n, 1)
- Каждый входной слой будет "прогоняться" через свой фильтр
- Далее все образоваашиеся слои будут "прогоняться" через Pointwise Convolution (точечная свертка)

Плюсы:
- Уменьшаем кол-во вычеслений (примерно в 9 раз!!!)
- Хорошо показываем зависимость соседних пикселей
- Мы учитываем зависимость между слоями

Минусы:
- Не заметил

То есть такой вариант свертки также хорош как и Depthwise Conv, однако он исключает его минусы. 
#### `Проверим на практике работоспособность такого варианта свертки`
----

### Типы сверток, которые мы будем проверять на практике:
     - Conv2D
     - DepthwiseConv2D
     - SeparableConv2D
- `Conv2DTranspose` (но не в этом ноуте)
----
----

# Pooling 
- Подобно сверточному слою, пулинговый слой необходим для уменьшения размера свернутого элемента в пространстве. Это помогает уменьшить вычислительную мощность, необходимую для обработки данных, за счет уменьшения размерности.
---

По аналогии со сверкой, мы отказываемся от 1D Pooling и 3D Pooling

----
### Average Pooling
[tf.keras.layers.AveragePooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AveragePooling2D)

![-](data/avgpool.jpg)

Отражает средние значения пикселей, из выборки pool_size
 
- Минимальое искажение изображения

----
### Max Pooling
[tf.keras.layers.MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)

![-](data/maxpool.png)

Показывает максимальное значения пикселей, из выборки pool_size

- Показывает выделяющиеся фичи, однако искажает изображение

----
### Global Average Pooling
[tf.keras.layers.GlobalAveragePooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling2D)

![-](data/globalavgpool.png)

Показывает средние значения пикселей, из входящего слоя

----
### Global Max Pooling
[tf.keras.layers.GlobalMaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalMaxPool2D)

![-](data/globalmaxpool.jpg)

Показывает средние значения пикселей, из входящего слоя

----
- Avg vs Max:

        Ничего сказать не могу, оба варианта стоит посмотреть на практике

- Base vs Global:

        Base уже показывал себя хорошо во время семинаров. Global будет работать плохо, потому что после одного такого пулинга, мы теряем колоссальное кол-во данных (остатется всего по 1 значению от каждого слоя)


----
### Типы пулингов, которые мы будем проверять на практике:
     - AveragePooling2D
     - MaxPool2D
----
----

# Комбинации convolutions и polling на практике.
----

## 1. Conv2D + AveragePooling2D

## 2. Conv2D + MaxPool2D

## 3. DepthwiseConv2D + AveragePooling2D

## 4. DepthwiseConv2D + MaxPool2D

## 5. SeparableConv2D + AveragePooling2D

## 6. SeparableConv2D + MaxPool2D